In [46]:
import pandas as pd
pd.options.display.max_rows = None

In [47]:
data = pd.read_csv('test_results.csv')
data.head()

,Date,Distance,Event / Venue,Joggler,Gender,Nationality,Finish Time,Drops,Notes / Result Links
0,23/05/2022,3b Mile,Time Trial,Henry Wellenstein,M,USA,00:04:39,0,NaN
1,01/02/2003,3b Mile,"Atlanta, Georgia",Will Howard,M,USA,00:04:42,?,http://www.atlantajugglers.org/aja-media/artic...
2,08/05/2018,3b Mile,Time Trial,Zach Prescott,M,USA,00:04:43,?,https://www.runnersworld.com/runners-stories/a...
3,1986,3b Mile,"IJA, San Jose",Kirk Swenson,M,USA,00:04:43,1,http://www.juggling.org/orgs/ija/champs.txt ht...
4,1985,3b Mile,"IJA, Atlanta",Kirk Swenson,M,USA,00:04:47,?,http://www.juggling.org/orgs/ija/champs.txt


In [56]:
def all_time_list(distance):
    fastest_times = data[data['Distance']==distance][['Joggler','Finish Time']].groupby(['Joggler']).min().reset_index()
    fastest_times = fastest_times.merge(data,how='left',left_on=['Joggler','Finish Time'],right_on=['Joggler','Finish Time'])
    fastest_times['Ranking'] = pd.to_numeric(fastest_times['Finish Time'].rank(method="min")).astype(int)
    fastest_times['Nationality'] = fastest_times['Nationality'].replace({'0':'Unknown'})
    fastest_times = fastest_times[['Ranking','Joggler','Gender','Nationality','Date','Event / Venue','Finish Time']].sort_values('Ranking').reset_index(drop=True)
    return fastest_times

all_time_list('5b Mile')

,Ranking,Joggler,Gender,Nationality,Date,Event / Venue,Finish Time
0,1,Matthew Feldman,M,USA,27/07/2012,Time Trial,00:06:33
1,2,Billy Gillen,M,USA,1989,Time Trial,00:07:41
2,3,Ben Schoenberg,M,Unknown,1996,"IJA, Rapid City",00:10:36
3,4,Barry Goldmeier,M,USA,1996,"IJA, Rapid City",00:10:37


In [66]:
pivot_df = pd.pivot_table(data,
                          values='Finish Time',
                          index='Joggler', 
                          columns='Distance', 
                          aggfunc='min')
pivot_df = pivot_df[['3b Mile','3b 5km','3b 10km','3b Half Marathon','3b Marathon','5b Mile']].reset_index().fillna('-')
pivot_df

Distance,Joggler,3b Mile,3b 5km,3b 10km,3b Half Marathon,3b Marathon,5b Mile
0,Aaron Scott,00:05:21,00:17:47,-,-,-,-
1,Adam Griffin,00:06:07,-,-,-,-,-
2,Albert Lucas,00:05:27,00:20:39,-,-,03:29:04,-
3,Andrea Goranson,00:10:56,-,-,-,-,-
4,Andrew Head,00:05:29,-,-,-,-,-
5,Andrew Hodge,-,00:29:01,-,-,-,-
6,Andrew Perrin,-,00:25:46,-,-,-,-
7,Andrew Ruiz,-,00:32:33,-,-,-,-
8,Andrew Swan,00:06:27,00:23:45,-,-,-,-
9,Andy Swan,-,00:23:21,-,-,-,-
